In [31]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [2]:
from skmine.itemsets import SLIM
from sklearn.datasets import fetch_20newsgroups

In [3]:
D = fetch_20newsgroups().data
D[:2]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

In [5]:
import dask.bag as db
X = db.from_sequence(D)
from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk import word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))



In [6]:
# stolen from https://github.com/screddy1313/Text-Classification-with-20news-dataset/blob/master/code/MultiClassNB.ipynb

def preprocessing(words):
    mod_words = []
    m_words = []
    symbols = ['.',',',':','?','!','<','>','(',')','#','--','-','$','@','%','``',';',"''"]
    for word in words:
        m_words.append(ps.stem(word))
    for word in m_words:
        if word not in stopWords:
            if word.isalpha():
                if len(word) > 2 :
                    mod_words.append(word.lower())
    return mod_words



In [7]:
X = X.str.replace("\n", "").str.split(" ").map(preprocessing)

In [8]:
_X = X.compute()

In [27]:
slim = SLIM(n_items=2000)

In [11]:
%%scalene
slim.fit(_X) 

In [28]:
%snakeviz -t slim.fit(_X)

 
*** Profile stats marshalled to file '/tmp/tmp45m6rryf'. 
Opening SnakeViz in a new tab...


In [32]:
slim.discover()

(articl, thi)      2927
(articl, would)     581
(like, would)      1649
(like, one)         637
(use, ani)         1521
                   ... 
(show, least)       175
(least, long)       111
(keep, let)         165
(chang, set)        164
(gener, let)         66
Length: 101, dtype: uint32

In [33]:
from skmine.preprocessing import SLIMTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

In [44]:
SLIMTransformer(k=10, strategy='one-hot').fit_transform(_X[:5])

,"(thi, realli)",(look),(univers),(wonder),"(articl, jonathan)","(could, engin)","(final, next)","(get, comput)","(got, know)","(onli, like)"
0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
4,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [37]:
MultiLabelBinarizer().fit_transform(_X)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [45]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

X_train = np.array(["new york is a hell of a town",
                "new york was originally dutch",
                "the big apple is great",
                "new york is also called the big apple",
                "nyc is nice",
                "people abbreviate new york city as nyc",
                "the capital of great britain is london",
                "london is in the uk",
                "london is in england",
                "london is in great britain",
                "it rains a lot in london",
                "london hosts the british museum",
                "new york is great and so is london",
                "i like london better than new york"])
y_train_text = [["new york"],["new york"],["new york"],["new york"],    
                ["new york"],["new york"],["london"],["london"],         
                ["london"],["london"],["london"],["london"],
                ["new york","England"],["new york","london"]]

X_test = np.array(['nice day in nyc',
               'welcome to london',
               'london is rainy',
               'it is raining in britian',
               'it is raining in britian and the big apple',
               'it is raining in britian and nyc',
               'hello welcome to new york. enjoy it here and london too'])

In [46]:
y_test_text = [["new york"],["new york"],["new york"],["new york"],["new york"],["new york"],["new york"]]


In [49]:

lb = preprocessing.MultiLabelBinarizer(classes=("new york","london","England"))
Y = lb.fit_transform(y_train_text)
Y_test = lb.fit_transform(y_test_text)

classifier = Pipeline([
('vectorizer', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(X_train, Y)
predicted = classifier.predict(X_test)

print("Accuracy Score: ",accuracy_score(Y_test, predicted))

Accuracy Score:  0.5714285714285714


In [51]:
from sklearn.preprocessing import FunctionTransformer

def s(text):
    return [t.split(" ") for t in text]

s(y_train_text)

AttributeError: 'list' object has no attribute 'split'